In [1]:
#Import requirements

import subprocess
import sys

import re
import time
import webbrowser
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
import networkx as nx
from datetime import datetime
from functools import reduce
import matplotlib.pyplot as plt
# import yahoo_fin.stock_info as si

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go



import warnings
warnings.filterwarnings("ignore")


In [2]:
main_file = pd.read_csv('merged_df.csv')
#Some text manipulation 
pattern = r"^(.*?): Close$"

# List to hold extracted ticker names
ticker_names = []
column_names = main_file.columns

#Matching the elements of column_names with pattern
for col in column_names:
    match = re.match(pattern, col)
    if match:
        ticker_names.append(match.group(1))  # group(1) refers to the first captured group

#Creating a list with target columns from main_file : "ticker_name: Close"
target_col_names = []
for ticker in ticker_names: 
    col_name = str(ticker + ': Close')
    target_col_names.append(col_name)

main_df = main_file[target_col_names].round(2)

#Creating new column names
# new_column_names = ['AAPL', 'MC.PA', 'JPM', 'NVDA', '^SPX', 'IAU', 'CL=F', 'LQD', 'XRP-USD',
#        'NVDA', 'LLY', 'TSLA', '^SPX','^DJI', '^RUT', 'IAU', 'CPER', 'GSG', 'CL=F', 'TLT', 
#                     'JNK', 'LQD', 'BTC-USD', 'ETH-USD', 'XRP-USD']

#Renaming columns
new_column_names = ['AAPL', 'META', 'V', 'MC.PA', 'NFLX', 'NKE', 'JPM', 'BAC', 'C',
                    'NVDA', 'LLY', 'TSLA', '^SPX', '^DJI', '^RUT', 'IAU', 'CPER',
                    'GSG', 'CL=F', 'TLT', 'JNK', 'LQD', 'BTC-USD', 'ETH-USD', 'XRP-USD']

#Setting main_file['Date'] as index of main_df    
main_df['Date'] = main_file['Date']
main_df.set_index(main_df['Date'], inplace = True)
pd.to_datetime(main_df.index, format='%Y-%m-%d')
main_df.drop('Date', axis=1, inplace=True)

# main_df.head()    #now we have in our main df columns of closing prices for each of our tickers. 
main_df.columns = new_column_names
# main_df

print(main_df.index.values[0:100])

['2017-11-09' '2017-11-10' '2017-11-13' '2017-11-14' '2017-11-15'
 '2017-11-16' '2017-11-17' '2017-11-20' '2017-11-21' '2017-11-22'
 '2017-11-24' '2017-11-27' '2017-11-28' '2017-11-29' '2017-11-30'
 '2017-12-01' '2017-12-04' '2017-12-05' '2017-12-06' '2017-12-07'
 '2017-12-08' '2017-12-11' '2017-12-12' '2017-12-13' '2017-12-14'
 '2017-12-15' '2017-12-18' '2017-12-19' '2017-12-20' '2017-12-21'
 '2017-12-22' '2017-12-27' '2017-12-28' '2017-12-29' '2018-01-02'
 '2018-01-03' '2018-01-04' '2018-01-05' '2018-01-08' '2018-01-09'
 '2018-01-10' '2018-01-11' '2018-01-12' '2018-01-16' '2018-01-17'
 '2018-01-18' '2018-01-19' '2018-01-22' '2018-01-23' '2018-01-24'
 '2018-01-25' '2018-01-26' '2018-01-29' '2018-01-30' '2018-01-31'
 '2018-02-01' '2018-02-02' '2018-02-05' '2018-02-06' '2018-02-07'
 '2018-02-08' '2018-02-09' '2018-02-12' '2018-02-13' '2018-02-14'
 '2018-02-15' '2018-02-16' '2018-02-20' '2018-02-21' '2018-02-22'
 '2018-02-23' '2018-02-26' '2018-02-27' '2018-02-28' '2018-03-01'
 '2018-03-

In [3]:
def debugging(df, ref_date, span):
    
    assert int(span) > 0, "span must be a positive integer"
    
    #REF_DATE DEBUGGING: 
    # Checking if the ref_date is in the df.index : if not, break all processes, 
    # and return ref_date to be the earliest
    if ref_date not in df.index:
        print('Ref_date not in df.index')
        ref_date = df.index[0]
    else: 
        ref_date = ref_date 
    
    #SPAN: 
    position = df.index.get_loc(ref_date) + 1
    if 2*span > position: 
        print('Span too high compared to index position. Rescaling')
        if position % 2 == 0: 
            span = position / 2
        else: 
            span = position // 2
    else: 
        span = span 
    
    span = int(span)
    print(f'Select position : {df.index[span]} minimum.')
    return ref_date, span


def rolling_corr(df, ref_date, span):

    ref_date, span = debugging(df, ref_date, span)

    try:
        position = df.index.get_loc(ref_date) + 1
        start_position = position - span
        filtered_df = df.iloc[start_position:position]
        corr_matrix = filtered_df.corr()

#         # Mask to zero out the upper triangle, including the diagonal
#         mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

#         # Apply the mask to the correlation matrix
#         lower_triangle_corr_matrix = corr_matrix.mask(mask)

#         return lower_triangle_corr_matrix.round(2)

        return corr_matrix.round(2)

    except KeyError as err: 
        print(f'Error due to wrong date/span input: {err}, Date : {ref_date}, Span : {span}.')
        print(f'Recall date range of input dataframe: {df.index[0], df.index[-1]}')

        
def relative_change(corr1, corr2):
    range_corr = 2
    rel_range_change = ((corr2 - corr1) / range_corr) * 100
    
    return rel_range_change        

In [4]:
def matrix_difference(matrix1, matrix2):

    # Check if the matrices have the same shape
    if matrix1.shape != matrix2.shape:
        raise ValueError("Matrices must have the same shape")
    
    # Initialize an empty matrix to store the differences
    rows = matrix1.shape[0]
    columns = matrix1.shape[1]
    result_matrix = pd.DataFrame(np.zeros((rows, columns)))
    result_matrix.columns = new_column_names
    
    # Iterate through the rows and columns of the matrices
    for i in range(rows):
        for j in range(columns):
            perct_change = relative_change(matrix1.iloc[i, j], matrix2.iloc[i, j])
            result_matrix.iloc[i, j] = perct_change
            
    return result_matrix



def matrix_difference_qual(matrix1, matrix2, heatmap=True):
    
    category_map = None

    # Check if the matrices have the same shape
    if matrix1.shape != matrix2.shape:
        raise ValueError("Matrices must have the same shape")
    
    # Initialize an empty matrix to store the differences
    rows = matrix1.shape[0]
    columns = matrix1.shape[1]
    result_matrix = pd.DataFrame(np.zeros((rows, columns)))
    result_matrix.columns = new_column_names
    
    # Iterate through the rows and columns of the matrices
    for i in range(rows):
        for j in range(columns):
            # Perform differentiation based on the values of the elements
            
            if matrix1.iloc[i, j] == 0 or matrix2.iloc[i, j] == 0:
                if matrix1.iloc[i, j] < 0:
                    matrix2.iloc[i,j] = -0.00001
                elif matrix1.iloc[i,j] > 0: 
                    matrix2.iloc[i,j] = 0.00001
                elif matrix2.iloc[i,j] < 0: 
                    matrix1.iloc[i,j] = 0.00001
                elif matrix2.iloc[i,j] > 0: 
                    matrix1.iloc[i,j] = -0.00001
                    
            if matrix1.iloc[i, j] < 0 and matrix2.iloc[i, j] < 0:
                if matrix2.iloc[i, j] > matrix1.iloc[i, j]:
                    result_matrix.iloc[i, j] = 'Neg Stronger'
                else:
                    result_matrix.iloc[i, j] = 'Neg Weaker'
            elif matrix1.iloc[i, j] > 0 and matrix2.iloc[i, j] > 0:
                if matrix2.iloc[i, j] > matrix1.iloc[i, j]:
                    result_matrix.iloc[i, j] = 'Pos Stronger'
                else:
                    result_matrix.iloc[i, j] = 'Pos Weaker'
            elif matrix1.iloc[i, j] > 0 and matrix2.iloc[i, j] < 0:
                result_matrix.iloc[i, j] = 'Neg Stronger'
            elif matrix1.iloc[i, j] < 0 and matrix2.iloc[i, j] > 0:
                result_matrix.iloc[i, j] = 'Pos Stronger'
            elif 0.95 <= matrix1.iloc[i, j] / matrix2.iloc[i, j] <= 1.05:
                result_matrix.iloc[i, j] = 'UNCH'
            else: 
                print(matrix1.iloc[i, j], matrix2.iloc[i,j])
                print(new_column_names[i], new_column_names[j])
                
    if heatmap: 
        category_map = {'Neg Stronger': -10, 'Neg Weaker': -5, 
                        'Pos Stronger': 10, 'Pos Weaker': 5, 
                        'UNCH': 0}
        df_numeric = result_matrix.applymap(lambda x: category_map[x])
        df_numeric.index = new_column_names
        
        return df_numeric

    else: 
        print("Can't return a heatmap - Categ Variables of String Type")
        return result_matrix

    
def rolling_corr_difference(df, ref_date, span):
    
    # Calculate correlation matrix for the current span
    corr_matrix_current = rolling_corr(df, ref_date, span)
    
    # Calculate the previous corr matrix's ref_date
    index_position = df.index.get_loc(ref_date)
    
    # Previous corr matrix's ref index position is max(0, index_position - span)
    temp_index_position = index_position - span 
    if temp_index_position < 0: 
        print(f'Period out of bound. Setting reference date to {df.index[0]}')
        temp_index_position = 0
        span = 0

    new_index_position = df.index.get_loc(df.index[temp_index_position])
    new_ref_date = df.index[new_index_position]

    # Calculate correlation matrix for the previous span
    corr_matrix_prev = rolling_corr(df, new_ref_date, span)
    
    # Calculate the difference between correlation matrices
    corr_diff = matrix_difference(corr_matrix_prev, corr_matrix_current)
    corr_diff_qual = matrix_difference_qual(corr_matrix_prev, corr_matrix_current)
    
    column_names = df.columns.tolist()

    mask = np.triu(np.ones_like(corr_diff, dtype=bool))
    corr_diff_masked = np.where(mask, None, corr_diff)  # Replace upper triangular part with None

    mask_qual = np.triu(np.ones_like(corr_diff_qual, dtype=bool))
    corr_diff_qual_masked = np.where(mask_qual, None, corr_diff_qual)  # Replace upper triangular part with None

    
    # Plotting heatmap for relative range percentage change
    fig_relative = go.Figure(data=go.Heatmap(z=corr_diff_masked, colorscale='RdYlGn',
                                             x=column_names, y=column_names))
    fig_relative.update_layout(title=f'Relative Range Percentage Change of Rolling Correlations between Assets Log Returns, {span} freq periods.',
                               xaxis_title='Assets', yaxis_title='Assets',                           
                               plot_bgcolor='white',  paper_bgcolor='white')

    # Assuming `corr_diff_qual` is plotted here instead
    fig_directional = go.Figure(data=go.Heatmap(z=corr_diff_qual_masked, colorscale='bluered',
                                                x=column_names, y=column_names))
    fig_directional.update_layout(title=f'Directional Difference between Rolling Correlation Matrices of Assets Log Returns, {span} freq periods.',
                                  xaxis_title='Assets', yaxis_title='Assets',
                                  plot_bgcolor='white',  paper_bgcolor='white')
    # Customizing the colorbar tick labels
    fig_directional.update_traces(colorbar_tickvals=[-10, -5, 0, 5, 10],
                  colorbar_ticktext=['Negative Stronger', 'Negative Weaker', 'UNCH', 'Positive Weaker', 'Positive Stronger'])
    
    fig_relative.update_layout(
    width=900,  # Adjust width
    height=900,  # Adjust height
    title=f'Relative Range Percentage Change of Rolling Correlations between Assets Log Returns, {span} freq periods.',
    xaxis_title='Assets',
    yaxis_title='Assets',
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis={'autorange': True, 'tickangle': 45},  # Rotate x-axis labels to prevent overlap
    yaxis={'autorange': True}
    )

    fig_directional.update_layout(
        width=900,  # Adjust width
        height=900,  # Adjust height
        title=f'Directional Difference between Rolling Correlation Matrices of Assets Log Returns, {span} freq periods.',
        xaxis_title='Assets',
        yaxis_title='Assets',
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis={'autorange': True, 'tickangle': 45},  # Rotate x-axis labels to prevent overlap
        yaxis={'autorange': True}
    )
    return fig_relative, fig_directional


# ref_date = '2024-02-01'
# rolling_corr_difference(main_df, ref_date, span=4)[0]
# rolling_corr_difference(main_df, ref_date, span)[1]

In [5]:
import plotly.graph_objs as go
import networkx as nx
import pandas as pd

def graph_net(df, ref_date, corr_threshold, span):
    corr_matrix = rolling_corr(df, ref_date, span)
    
    # Initialize the graph
    G = nx.Graph()
    
    # Ticker categories and their colors
    ticker_categs = {
        'Equity': ['AAPL', 'META', 'V', 'MC.PA', 'NFLX', 'NKE', 'JPM', 'BAC', 'C', 'NVDA', 'LLY', 'TSLA'],
        'Index': ['^SPX', '^DJI', '^RUT'],
        'Credit': ['TLT', 'JNK', 'LQD'],
        'Commodities': ['IAU', 'CPER', 'GSG', 'CL=F'],
        'Crypto': ['BTC-USD', 'ETH-USD', 'XRP-USD']
    }
    
    colors = {
        'Equity': 'yellow',
        'Index': 'blue',
        'Credit': 'red',
        'Commodities': 'orange',
        'Crypto': 'green'
    }
    
    # Node colors based on their category
    node_colors = {}
    for category, nodes in ticker_categs.items():
        for node in nodes:
            node_colors[node] = colors[category]
    
    # Determine which nodes should be included based on the correlation threshold
    nodes_to_include = set()
    for i in corr_matrix.columns:
        for j in corr_matrix.columns:
            if i != j and abs(corr_matrix.loc[i, j]) > corr_threshold:
                nodes_to_include.add(i)
                nodes_to_include.add(j)
    
    # Only add nodes that are part of an edge meeting the threshold
    for node in nodes_to_include:
        G.add_node(node, color=node_colors.get(node, 'grey'))
    
    # Add edges to the graph based on correlation
    for i in corr_matrix.columns:
        for j in corr_matrix.columns:
            if i != j:  # Ensure we don't compare the same stock to itself
                corr = corr_matrix.loc[i, j]
                if abs(corr) > corr_threshold:  # Check if the correlation meets the threshold
                    # Add an edge with color based on the sign of the correlation
                    G.add_edge(i, j, weight=corr, color='green' if corr > 0 else 'red')

    # Assuming 'G' is your original graph with 'weight' attributes holding the correlations
    H = G.copy()

    # Update edge weights in H to be absolute values of the original weights
    for u, v, d in H.edges(data=True):
        d['weight'] = abs(d['weight'])


    # Assuming H is your graph for layout and G contains original correlation weights
    pos = nx.kamada_kawai_layout(H)                    
    

    # Initialize the figure once, before the loop
    fig = go.Figure()

    # For edges, create individual traces within the loop
    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        corr_value = edge[2]['weight']

        # Determine the color based on the correlation value
        edge_color = 'green' if corr_value > 0 else 'red'

        # Create an individual trace for this edge
        edge_trace = go.Scatter(
            x=[x0, x1, None], 
            y=[y0, y1, None],
            line=dict(width=0.5, color=edge_color),
            mode='lines',
            hoverinfo='none',
            showlegend=False# No hover info for the line itself
        )
        fig.add_trace(edge_trace)

        # Invisible marker at the midpoint for hover text
        midpoint_trace = go.Scatter(
            x=[(x0 + x1) / 2],
            y=[(y0 + y1) / 2],
            text=[f'{edge[0]}-{edge[1]}: {corr_value:.2f}'],
            mode='markers',
            hoverinfo='text',
            marker=dict(size=0.1, color='rgba(0,0,0,0)'),  # Make the marker virtually invisible
            showlegend=False
        )
        fig.add_trace(midpoint_trace)

        
    # Track which categories have been added to the legend
    added_categories = set()

    for node in G.nodes():
        x, y = pos[node]
        category = None
        for categ, members in ticker_categs.items():
            if node in members:
                category = categ
                break
        if category and category not in added_categories:
            # Add a representative node for this category to the legend
            fig.add_trace(go.Scatter(
                x=[x], y=[y],
                mode='markers+text',
                marker=dict(color=colors[category], size=10),
                name=category,  # This sets the legend entry,
                hoverinfo='none'
            ))
            added_categories.add(category)

    # Add node trace after all edge traces have been added
    node_x = []
    node_y = []
    node_text = []
    node_marker_colors = []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(node)
        node_marker_colors.append(G.nodes[node]['color'])

    node_trace = go.Scatter(
        x=node_x, y=node_y, text=node_text, mode='markers+text', hoverinfo='none',
        marker=dict(showscale=False, color=node_marker_colors, size=20, line_width=2),
        textposition="bottom center", showlegend=True
    )

    fig.add_trace(node_trace)

    # Set the layout for the figure
    fig.update_layout(
        showlegend=True,
        hovermode='closest',
        margin=dict(b=0,l=0,r=0,t=0),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        legend_title_text='Node Categories',
        legend=dict(x=1, y=0, xanchor='right', yanchor='bottom')
    )

    return fig


In [6]:
#Global Vars
x = 0.9
ref_date = '2024-02-14'
span = 2
new_column_names = ['AAPL', 'META', 'V', 'MC.PA', 'NFLX', 'NKE', 'JPM', 'BAC', 'C',
                    'NVDA', 'LLY', 'TSLA', '^SPX', '^DJI', '^RUT', 'IAU', 'CPER',
                    'GSG', 'CL=F', 'TLT', 'JNK', 'LQD', 'BTC-USD', 'ETH-USD', 'XRP-USD']

#for dash app
dates_list = main_df.index.tolist()
ref_date = dates_list[24]
dash_df = main_df

# graph_net(main_df, ref_date, x, span)
print(dates_list[0])
print(dates_list[24])
debugging(main_df, ref_date, span)

2017-11-09
2017-12-14
Select position : 2017-11-13 minimum.


('2017-12-14', 2)

In [7]:
# import dash
# from dash import dcc, html
# from dash.dependencies import Input, Output
# import pandas as pd
# import plotly.graph_objs as go
# import networkx as nx

# # Assume 'df' is your DataFrame and 'graph_net' is your function
# # Your DataFrame 'df' needs to be loaded before this point
# df = main_df
# span = 90
# # Initialize Dash app
# app = dash.Dash(__name__)

# # Get the list of reference dates excluding the first 11 dates
# available_dates = df.index.tolist()

# # Define app layout with a flex container for alignment
# app.layout = html.Div([
#     html.H1("Some plots", style={'textAlign': 'center'}),
#     html.H2("Network Graph Visualization", style={'textAlign':'center'}),
#     html.Div([
#         html.Div([
#             html.Label("Correlation Threshold", style={'textAlign': 'center'}),
#             dcc.Slider(
#                 id='corr-threshold-slider',
#                 min=0,
#                 max=1,
#                 step=0.01,
#                 value=0.5,  # Default value
#                 marks={str(i/10): str(i / 10) for i in range(0, 11)}, 
#             )
#         ], style={'width': '45%', 'display': 'inline-block', 'padding': '20px'}),

#         html.Div([
#             dcc.DatePickerSingle(
#                 id='date-picker',
#                 min_date_allowed=min(available_dates),
#                 max_date_allowed=max(available_dates),
#                 initial_visible_month=min(available_dates),
#                 date=str(min(available_dates))  # Set initial date
#             )
#         ], style={'width': '45%', 'display': 'inline-block', 'textAlign': 'right', 'float': 'right', 'padding': '20px'}),
#     ], style={'display': 'flex', 'justifyContent': 'space-between'}),
    
#     dcc.Graph(id='network-graph'),
#     html.H2("Heatmap, Relative", style={'textAlign': 'center'}),
#     dcc.Graph(id='heatmap-relative'),
#     html.H2("Heatmap, Directional", style={'textAlign': 'center'}),
#     dcc.Graph(id='heatmap-directional'),
    
#     html.Div(id='error-message', style={'color': 'red', 'fontWeight': 'bold'})  # Error message div
# ], style={'padding': '20px'})


# # Callback to update graph based on selected date and correlation threshold
# @app.callback(
#     [Output('network-graph', 'figure'),
#      Output('heatmap-relative', 'figure'),
#      Output('heatmap-directional', 'figure'),
#      Output('error-message', 'children')],
#     [Input('date-picker', 'date'),
#      Input('corr-threshold-slider', 'value')]
# )

# def update_graph(selected_date, corr_threshold):
#     if selected_date is not None and corr_threshold is not None:
#         # Check if selected_date is valid using rolling_corr or equivalent function
#         corr_result = rolling_corr(df, selected_date, span=span)  # Adjust span if necessary
#         if isinstance(corr_result, str) and corr_result == "Date not found":
#             return dash.no_update, "Error: Selected date not found in the dataset."
#         else:
#             try: 
#                 network_fig = graph_net(df, selected_date, corr_threshold=corr_threshold, span=span)
#                 heatmap_relative_graph, heatmap_directional_graph = rolling_corr_difference(df, selected_date, span=span)
#                 return network_fig, heatmap_relative_graph, heatmap_directional_graph, ""
            
#             except Exception as e:
#                 return dash.no_update, dash.no_update, dash.no_update, f"Error: {str(e)}"
#     else:
#         return dash.no_update, dash.no_update, dash.no_update, "" 


# # Run the app
# port = 8080

# # Open a web browser tab using the specified port
# def open_browser():
#       webbrowser.open_new_tab(f'http://127.0.0.1:{port}')

# if __name__ == '__main__':
#     # Use the threading module to open a web browser tab
#     # This prevents blocking the execution of the app
#     from threading import Timer
#     Timer(1, open_browser).start()  # Wait 1 second before opening the tab
    
#     # Run the app
#     app.run_server(debug=True, port=port)


In [8]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import networkx as nx

# Assume 'df' is your DataFrame and 'graph_net' is your function
# Your DataFrame 'df' needs to be loaded before this point
df = main_df
span = 90
# Initialize Dash app
app = dash.Dash(__name__)

# Get the list of reference dates excluding the first 11 dates
available_dates = df.index.tolist()

# Define app layout with a flex container for alignment
app.layout = html.Div([
    html.H1("Some plots", style={'textAlign': 'center'}),
    html.H2("Network Graph Visualization", style={'textAlign':'center'}),
    html.Div([
        html.Div([
            html.Label("Correlation Threshold", style={'textAlign': 'center'}),
            dcc.Slider(
                id='corr-threshold-slider',
                min=0,
                max=1,
                step=0.01,
                value=0.5,  # Default value
                marks={str(i/10): str(i / 10) for i in range(0, 11)}, 
            )
        ], style={'width': '45%', 'display': 'inline-block', 'padding': '20px'}),

        html.Div([
            dcc.DatePickerSingle(
                id='date-picker',
                min_date_allowed=min(available_dates),
                max_date_allowed=max(available_dates),
                initial_visible_month=min(available_dates),
                date=str(min(available_dates))  # Set initial date
            )
        ], style={'width': '45%', 'display': 'inline-block', 'textAlign': 'right', 'float': 'right', 'padding': '20px'}),
    ], style={'display': 'flex', 'justifyContent': 'space-between'}),
    


    dcc.Graph(id='network-graph'),
    html.Div([
        html.Div([
            html.H2("Heatmap, Relative", style={'textAlign': 'center'}),
            dcc.Graph(id='heatmap-relative'),
        ], style={'width': '50%', 'display': 'inline-block'}),

        html.Div([
            html.H2("Heatmap, Directional", style={'textAlign': 'center'}),
            dcc.Graph(id='heatmap-directional'),
        ], style={'width': '50%', 'display': 'inline-block'}),
    ], style={'display': 'flex'}),  # This container will hold both heatmaps side by side
    
    html.Div(id='error-message', style={'color': 'red', 'fontWeight': 'bold'})  # Error message div
], style={'padding': '20px'})


# Callback to update graph based on selected date and correlation threshold
@app.callback(
    [Output('network-graph', 'figure'),
     Output('heatmap-relative', 'figure'),
     Output('heatmap-directional', 'figure'),
     Output('error-message', 'children')],
    [Input('date-picker', 'date'),
     Input('corr-threshold-slider', 'value')]
)

def update_graph(selected_date, corr_threshold):
    if selected_date is not None and corr_threshold is not None:
        # Check if selected_date is valid using rolling_corr or equivalent function
        corr_result = rolling_corr(df, selected_date, span=span)  # Adjust span if necessary
        if isinstance(corr_result, str) and corr_result == "Date not found":
            return dash.no_update, "Error: Selected date not found in the dataset."
        else:
            try: 
                network_fig = graph_net(df, selected_date, corr_threshold=corr_threshold, span=span)
                heatmap_relative_graph, heatmap_directional_graph = rolling_corr_difference(df, selected_date, span=span)
                return network_fig, heatmap_relative_graph, heatmap_directional_graph, ""
            
            except Exception as e:
                return dash.no_update, dash.no_update, dash.no_update, f"Error: {str(e)}"
    else:
        return dash.no_update, dash.no_update, dash.no_update, "" 


# Run the app
port = 8080

# Open a web browser tab using the specified port
def open_browser():
      webbrowser.open_new_tab(f'http://127.0.0.1:{port}')

if __name__ == '__main__':
    # Use the threading module to open a web browser tab
    # This prevents blocking the execution of the app
    from threading import Timer
    Timer(1, open_browser).start()  # Wait 1 second before opening the tab
    
    # Run the app
    app.run_server(debug=True, port=port)


In [9]:
# from flask import Flask, render_template, request, send_file


# app = Flask(__name__)


# @app.route('/')
# def index():
#     available_dates = dates_list[11:]
#     return render_template('index.html', available_dates=available_dates)

# @app.route('/plot', methods=['POST'])
# def plot():
#     selected_date = request.form.get('selected_date')
#     corr_threshold = 0.5  # Adjust as needed
#     span = 5  # Fixed span
    
#     plot_buf = graph_net(dash_df, ref_date, corr_threshold, span)
    
#     return send_file(plot_buf, mimetype='image/png')

# if __name__ == '__main__':
#     app.run(port=5001)
#     app.run(debug=True)


In [10]:
# TRYING TO MAKE NODES & EDGES SELECTION BETTER 

# import plotly.graph_objs as go
# import networkx as nx
# import pandas as pd

# def graph_net(df, ref_date, corr_threshold, span, selected_categories=ticker_categs.keys()):
#     global ticker_categs
    
#     corr_matrix = rolling_corr(df, ref_date, span)
#     selected_categs = selected_categories
    
#     # Initialize the graph
#     G = nx.Graph()
    
#     # Ticker categories and their colors
#     ticker_categs = {
#         'Equity': ['AAPL', 'META', 'V', 'MC.PA', 'NFLX', 'NKE', 'JPM', 'BAC', 'C', 'NVDA', 'LLY', 'TSLA'],
#         'Index': ['^SPX', '^DJI', '^RUT'],
#         'Credit': ['TLT', 'JNK', 'LQD'],
#         'Commodities': ['IAU', 'CPER', 'GSG', 'CL=F'],
#         'Crypto': ['BTC-USD', 'ETH-USD', 'XRP-USD']
#     }
    
#     colors = {
#         'Equity': 'yellow',
#         'Index': 'blue',
#         'Credit': 'red',
#         'Commodities': 'orange',
#         'Crypto': 'green'
#     }
    
#     selected_nodes = [node for category in selected_categs for node in ticker_categs[category]]
#     # selected_nodes = [node for category in selected_categories for node in ticker_categs[category]]
    
#     # Node colors based on their category
#     node_colors = {}
#     for category, nodes in ticker_categs.items():
#         for node in nodes:
#             node_colors[node] = colors[category]
    
#     # Determine which nodes should be included based on the correlation threshold
#     nodes_to_include = set()
#     for i in corr_matrix.columns:
#         for j in corr_matrix.columns:
#             if i != j and abs(corr_matrix.loc[i, j]) > corr_threshold:
#                 nodes_to_include.add(i)
#                 nodes_to_include.add(j)
    
#     # Only add nodes that are part of an edge meeting the threshold
#     for node in nodes_to_include:
#         G.add_node(node, color=node_colors.get(node, 'grey'))
    
#     # Add edges to the graph based on correlation and selection : 
#     for i in corr_matrix.columns:
#         for j in corr_matrix.columns:
#             if i != j:  # Ensure we don't compare the same stock to itself
#                 corr = corr_matrix.loc[i, j]
#                 if abs(corr) > corr_threshold:  # Check if the correlation meets the threshold
#                     # Add an edge with color based on the sign of the correlation
#                     G.add_edge(i, j, weight=corr, color='green' if corr > 0 else 'red')

#     # Assuming 'G' is your original graph with 'weight' attributes holding the correlations
#     H = G.copy()

#     # Update edge weights in H to be absolute values of the original weights
#     for u, v, d in H.edges(data=True):
#         d['weight'] = abs(d['weight'])


#     # Assuming H is your graph for layout and G contains original correlation weights
#     pos = nx.kamada_kawai_layout(H)                    
    

#     # Initialize the figure once, before the loop
#     fig = go.Figure()

#     # For edges, create individual traces within the loop
#     for edge in G.edges(data=True):
#         # Check if both nodes of the edge are in the selected nodes
#         if edge[0] in selected_nodes and edge[1] in selected_nodes:
#             x0, y0 = pos[edge[0]]
#             x1, y1 = pos[edge[1]]
#             corr_value = edge[2]['weight']

#             # Determine the color based on the correlation value
#             edge_color = 'green' if corr_value > 0 else 'red'

#             # Create an individual trace for this edge
#             edge_trace = go.Scatter(
#                 x=[x0, x1, None], 
#                 y=[y0, y1, None],
#                 line=dict(width=0.5, color=edge_color),
#                 mode='lines',
#                 hoverinfo='none',
#                 showlegend=False# No hover info for the line itself
#             )
#             fig.add_trace(edge_trace)

#             # Invisible marker at the midpoint for hover text
#             midpoint_trace = go.Scatter(
#                 x=[(x0 + x1) / 2],
#                 y=[(y0 + y1) / 2],
#                 text=[f'{edge[0]}-{edge[1]}: {corr_value:.2f}'],
#                 mode='markers',
#                 hoverinfo='text',
#                 marker=dict(size=0.1, color='rgba(0,0,0,0)'),  # Make the marker virtually invisible
#                 showlegend=False
#             )
#             fig.add_trace(midpoint_trace)

        
#     for category, members in ticker_categs.items():
#         node_x = []
#         node_y = []
#         node_text = []
#         node_marker_colors = []

#         for node in G.nodes():
#             if node in members and list(G.edges(node)):
#                 x, y = pos[node]
#                 node_x.append(x)
#                 node_y.append(y)
#                 node_text.append(node)
#                 node_marker_colors.append(colors[category])

#         node_trace = go.Scatter(
#             x=node_x, y=node_y, text=node_text, mode='markers+text', hoverinfo='none',
#             marker=dict(showscale=False, color=node_marker_colors, size=20, line_width=2),
#             textposition="bottom center",
#             name=category,
#         )

#         fig.add_trace(node_trace)

#     # Set the layout for the figure
#     fig.update_layout(
#         showlegend=True,
#         hovermode='closest',
#         margin=dict(b=0,l=0,r=0,t=0),
#         xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
#         yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
#         legend_title_text='Node Categories',
#         legend=dict(x=1, y=0, xanchor='right', yanchor='bottom')
#     )

#     return fig


# import dash
# from dash import dcc, html
# from dash.dependencies import Input, Output
# import pandas as pd
# import plotly.graph_objs as go
# import networkx as nx

# # Assume 'df' is your DataFrame and 'graph_net' is your function
# # Your DataFrame 'df' needs to be loaded before this point
# df = main_df
# # Initialize Dash app
# app = dash.Dash(__name__)

# # Get the list of reference dates excluding the first 11 dates
# available_dates = df.index.tolist()[11:]

# # Define app layout with a flex container for alignment
# app.layout = html.Div([
#     html.H1("Network Graph Visualization", style={'textAlign': 'center'}),
#     html.Div([
#         html.Div([
#             html.Label("Correlation Threshold", style={'textAlign': 'center'}),
#             dcc.Slider(
#                 id='corr-threshold-slider',
#                 min=0,
#                 max=1,
#                 step=0.05,
#                 value=0.5,  # Default value
#                 marks={i / 10: str(i / 10) for i in range(0, 11, 1)},
#             )
#         ], style={'width': '45%', 'display': 'inline-block', 'padding': '20px'}),

#         html.Div([
#             dcc.DatePickerSingle(
#                 id='date-picker',
#                 min_date_allowed=min(available_dates),
#                 max_date_allowed=max(available_dates),
#                 initial_visible_month=min(available_dates),
#                 date=str(min(available_dates))  # Set initial date
#             )
#         ], style={'width': '45%', 'display': 'inline-block', 'textAlign': 'right', 'float': 'right', 'padding': '20px'}),
#     ], style={'display': 'flex', 'justifyContent': 'space-between'}),
#     dcc.Graph(id='network-graph'),
    
#     html.Div([
#         dcc.Checklist(
#             id='category-selector',
#             options=[{'label': category, 'value': category} for category in ticker_categs.keys()],
#             value=list(ticker_categs.keys()),  # Default all selected
#             labelStyle={'display': 'block'}
#         )
#     ]),
    
#     html.Div(id='error-message', style={'color': 'red', 'fontWeight': 'bold'})  # Error message div
# ], style={'padding': '20px'})


# # Callback to update graph based on selected date and correlation threshold
# @app.callback(
#     [Output('network-graph', 'figure'),
#      Output('error-message', 'children')],
#     [Input('date-picker', 'date'),
#      Input('corr-threshold-slider', 'value'),
#      Input('category-selector', 'value')]
# )

# def update_graph(selected_date, corr_threshold, selected_categories):
#     if selected_date is not None and corr_threshold is not None:
#         # Check if selected_date is valid using rolling_corr or equivalent function
#         corr_result = rolling_corr(df, selected_date, span=3)  # Adjust span if necessary
#         if isinstance(corr_result, str) and corr_result == "Date not found":
#             return dash.no_update, "Error: Selected date not found in the dataset."
#         else:
#             fig = graph_net(df, selected_date, corr_threshold=corr_threshold, span=3,selected_categories=selected_categories)
#             return fig, ""
#     else:
#         return go.Figure(), ""  # Return empty graph and no error message if no date or threshold is selected


# # Run the app
# port = 8050

# # Open a web browser tab using the specified port
# def open_browser():
#       webbrowser.open_new_tab(f'http://127.0.0.1:{port}')

# if __name__ == '__main__':
#     # Use the threading module to open a web browser tab
#     # This prevents blocking the execution of the app
#     from threading import Timer
#     Timer(1, open_browser).start()  # Wait 1 second before opening the tab
    
#     # Run the app
#     app.run_server(debug=True, port=port)



In [11]:
# ###WITH ONLY NET GRAPH


# df = main_df
# # Initialize Dash app
# app = dash.Dash(__name__)

# # Get the list of reference dates excluding the first 11 dates
# available_dates = df.index.tolist()[11:]

# # Define app layout with a flex container for alignment
# app.layout = html.Div([
#     html.H1("Some plots", style={'textAlign': 'center'}),
#     html.H2("Network Graph Visualization", style={'textAlign':'center'}),
#     html.Div([
#         html.Div([
#             html.Label("Correlation Threshold", style={'textAlign': 'center'}),
#             dcc.Slider(
#                 id='corr-threshold-slider',
#                 min=0,
#                 max=1,
#                 step=0.05,
#                 value=0.5,  # Default value
#                 marks={str(i / 10): str(i / 10) for i in range(0, 11)},  # Key change: Convert keys to string
#             )
#         ], style={'width': '45%', 'display': 'inline-block', 'padding': '20px'}),

#         html.Div([
#             dcc.DatePickerSingle(
#                 id='date-picker',
#                 min_date_allowed=min(available_dates),
#                 max_date_allowed=max(available_dates),
#                 initial_visible_month=min(available_dates),
#                 date=str(min(available_dates))  # Set initial date
#             )
#         ], style={'width': '45%', 'display': 'inline-block', 'textAlign': 'right', 'float': 'right', 'padding': '20px'}),
#     ], style={'display': 'flex', 'justifyContent': 'space-between'}),
    
#     dcc.Graph(id='network-graph'),
#     html.H2("Heatmap, Relative", style={'textAlign': 'center'}),
#     html.Div(id='error-message', style={'color': 'red', 'fontWeight': 'bold'})  # Error message div
# ], style={'padding': '20px'})


# # Callback to update graph based on selected date and correlation threshold
# @app.callback(
#     [Output('network-graph', 'figure'),
#      Output('error-message', 'children')],
#     [Input('date-picker', 'date'),
#      Input('corr-threshold-slider', 'value')]
# )

# def update_graph(selected_date, corr_threshold):
#     if selected_date is not None and corr_threshold is not None:
#         # Check if selected_date is valid using rolling_corr or equivalent function
#         corr_result = rolling_corr(df, selected_date, span=3)  # Adjust span if necessary
#         if isinstance(corr_result, str) and corr_result == "Date not found":
#             return dash.no_update, "Error: Selected date not found in the dataset."
#         else:
#             try: 
#                 network_fig = graph_net(df, selected_date, corr_threshold=corr_threshold, span=3)
#                 return network_fig, ""
            
#             except Exception as e:
#                 return dash.no_update, f"Error: {str(e)}"
#     else:
#         return dash.no_update, ""


# # Run the app
# port = 8050

# # Open a web browser tab using the specified port
# def open_browser():
#       webbrowser.open_new_tab(f'http://127.0.0.1:{port}')

# if __name__ == '__main__':
#     # Use the threading module to open a web browser tab
#     # This prevents blocking the execution of the app
#     from threading import Timer
#     Timer(1, open_browser).start()  # Wait 1 second before opening the tab
    
#     # Run the app
#     app.run_server(debug=True, port=port)


Span too high compared to index position. Rescaling
Select position : 2017-11-09 minimum.
Span too high compared to index position. Rescaling
Select position : 2017-11-09 minimum.
Span too high compared to index position. Rescaling
Select position : 2017-11-09 minimum.
Period out of bound. Setting reference date to 2017-11-09
Span too high compared to index position. Rescaling
Select position : 2017-11-09 minimum.
Span too high compared to index position. Rescaling
Select position : 2017-11-09 minimum.
Span too high compared to index position. Rescaling
Select position : 2017-11-09 minimum.
Period out of bound. Setting reference date to 2017-11-09
Span too high compared to index position. Rescaling
Select position : 2018-03-16 minimum.
Span too high compared to index position. Rescaling
Select position : 2018-03-16 minimum.
Span too high compared to index position. Rescaling
Select position : 2018-03-16 minimum.
Span too high compared to index position. Rescaling
Select position : 2018